In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df= spark.read.format("parquet").option("header","true").option("inferSchema","true").load(f"abfss://bronze@olympicsprojdatalk.dfs.core.windows.net/athletes")

In [0]:
display(df)

In [0]:
df = df.fillna({'birth_place': 'Unknown', 'birth_country': 'Unknown', 'residence_place': 'Unknown', 'residence_country': 'Unknown'})

In [0]:
df.display()

In [0]:
df_filter = df.filter(col("current")== True & col("name").isin("GALSTYAN Slavik", "SEHEN Sajjad"))

In [0]:
df_filter.display()

In [0]:
%python
df = df.withColumn('height', col('height').cast(FloatType())).withColumn('weight', col('weight').cast(FloatType()))

In [0]:
display(df)

In [0]:
df_sort = df.sort("height","weight",ascending=[0,1]).filter(col("weight") > 0)

In [0]:
df_sort.display()

In [0]:
df_sort = df_sort.withColumn("nationality", regexp_replace("nationality", "united states", "us"))

In [0]:
df.groupBy("code").agg(count("code").alias("total_count")).filter(col("total_count") > 1).display()

In [0]:
df_sort = df_sort.withColumnRenamed("code", "athlete_id")

In [0]:
df_sort = df_sort.withColumn('occupation',split(col('occupation'),','))

In [0]:
df_sort.display()

In [0]:
df_sort.columns

In [0]:
df_final = df_sort.select('athlete_id',
 'current',
 'name',
 'name_short',
 'name_tv',
 'gender',
 'function',
 'country_code',
 'country',
 'country_long',
 'nationality',
 'nationality_long',
 'nationality_code',
 'height',
 'weight')

In [0]:
display(df_final)

Databricks visualization. Run in Databricks to view.

In [0]:
df_final.write.format("delta").mode("append").option("path","abfss://silver@olympicsprojdatalk.dfs.core.windows.net/athletes").saveAsTable("olympics.silver.athletes")